# Concordance Correlation Coefficient and Pitman-Morgan Test
## Uploaded on https://github.com/krg-uoi

In [1]:
import numpy as np

def concordance_correlation_coefficient(y_true, y_pred, sample_weight=None, multioutput='uniform_average'):
    """Lin’s concordance correlation coefficient (CCC) is the concordance between a new test or measurement (Y) and a
gold standard test or measurement (X). This statistic quantifies the agreement between these two measures of the
same variable being a measure of inter-rater agreement.
    Original paper: Lawrence, I., and Kuei Lin. "A concordance correlation coefficient to evaluate reproducibility." Biometrics (1989): 255-268.  
    
    The Comprehensive R Archive Network: https://cran.r-project.org/web/packages/epiR/epiR.pdf
    
    Parameters
    ----------
    y_true : array-like of shape = (n_samples) or (n_samples, n_outputs)
        Ground truth (correct) target values.
    y_pred : array-like of shape = (n_samples) or (n_samples, n_outputs)
        Estimated target values.
    Returns
    -------
    loss : A float in the range [-1,1]. A value of 1 indicates perfect agreement
    between the true and the predicted values.
    Examples
    --------
    >>> from sklearn.metrics import concordance_correlation_coefficient
    >>> y_true = [3, -0.5, 2, 7]
    >>> y_pred = [2.5, 0.0, 2, 8]
    >>> concordance_correlation_coefficient(y_true, y_pred)
    0.97678916827853024
    """
    cor=np.corrcoef(y_true,y_pred)[0][1]
    
    mean_true=np.mean(y_true)
    mean_pred=np.mean(y_pred)
    
    var_true=np.var(y_true)
    var_pred=np.var(y_pred)
    
    sd_true=np.std(y_true)
    sd_pred=np.std(y_pred)
    
    numerator=2*cor*sd_true*sd_pred
    
    denominator=var_true+var_pred+(mean_true-mean_pred)**2

    return numerator/denominator

In [27]:
import random

n_samples=100
y_true = np.arange(n_samples) +1 
y_pred = y_true * random.random()
print (y_pred)

c=concordance_correlation_coefficient(y_true,y_pred)
print("CCC=", c)

[ 0.12706724  0.25413449  0.38120173  0.50826897  0.63533622  0.76240346
  0.8894707   1.01653795  1.14360519  1.27067243  1.39773968  1.52480692
  1.65187416  1.77894141  1.90600865  2.03307589  2.16014314  2.28721038
  2.41427763  2.54134487  2.66841211  2.79547936  2.9225466   3.04961384
  3.17668109  3.30374833  3.43081557  3.55788282  3.68495006  3.8120173
  3.93908455  4.06615179  4.19321903  4.32028628  4.44735352  4.57442076
  4.70148801  4.82855525  4.95562249  5.08268974  5.20975698  5.33682422
  5.46389147  5.59095871  5.71802595  5.8450932   5.97216044  6.09922768
  6.22629493  6.35336217  6.48042941  6.60749666  6.7345639   6.86163115
  6.98869839  7.11576563  7.24283288  7.36990012  7.49696736  7.62403461
  7.75110185  7.87816909  8.00523634  8.13230358  8.25937082  8.38643807
  8.51350531  8.64057255  8.7676398   8.89470704  9.02177428  9.14884153
  9.27590877  9.40297601  9.53004326  9.6571105   9.78417774  9.91124499
 10.03831223 10.16537947 10.29244672 10.41951396 10.

In [28]:
import numpy as np
from scipy.stats import t


def PitmanMorganTest(x, y):
    """Pitman-Morgan Test for the difference between correlated variances with
    paired samples.

    Code adapted to Python from Janne Kauttonen's MatLab PitmanMorganTest(x,y)
    function: https://www.mathworks.com/matlabcentral/fileexchange/67910-pitmanmorgantest-x-y
    which in return is copied from a Excel sheet in "how2stats" blog posting:
    http://www.how2stats.net/2011/06/testing-difference-between-correlated.html
    which is based on Gardner, R.C. (2001). Psychological Statistics Using SPSS
    for Windows. New Jersey: Prentice Hall

    The following functions can be used to replicate matlab's corr2 function, which
    returns the correlation coefficient as a single number, in contrast to
    numpy's corcoeff, which returns the correlation coefficient matrix.
    Check: https://stackoverflow.com/questions/29481518/python-equivalent-of-matlab-corr2

        def mean2(x):
            y = np.sum(x) / np.size(x)
            return y

        def corr2(a, b):
            a = a - mean2(a)
            b = b - mean2(b)

            r = (a*b).sum() / np.sqrt((a*a).sum() * (b*b).sum())
            return r

    Args:
        x (array-like): sample vector x.
        y (array-like): sample vector y.

    Raises:
        ValueError: If `x` and `y` have different lengths.

    Returns:
        A tuple (h, p, ratio).

        h (bool): If False, the null hypothesis ("same variance") cannot be
        rejected at the 5% significance level.

        p (float): Test statistics (two-tailed).

        ratio (float): Variance ratio. Computed from var[0]/var[1], where
        var[0] >= var[1].

    """
    N = len(x)

    if len(y) != N:
        raise ValueError("Length mismatch between 'x' and 'y' arrays.")

    vars = [np.var(x), np.var(y)]

    cor = np.corrcoef(x, y)[0][1]

    # must have vars[0] > var[1]
    if vars[0] < vars[1]:
        vars[0], vars[1] = vars[1], vars[0]

    ratio = vars[0] / vars[1]

    # formulas from Garder (2001, p.57)
    numerator1_S1minusS2 = vars[0] - vars[1]
    numerator2_SQRTnminus2 = np.sqrt(N - 2)
    numerator3 = numerator1_S1minusS2 * numerator2_SQRTnminus2
    denominator1_4timesS1timesS2 = 4 * vars[0] * vars[1]
    denominator2_rSquared = cor**2
    denominator3_1minusrSquared = 1 - denominator2_rSquared
    denominator4_4timesS1timesS2div1minusrSquared = denominator1_4timesS1timesS2 * \
        denominator3_1minusrSquared
    denominator5 = np.sqrt(denominator4_4timesS1timesS2div1minusrSquared)
    df = N - 2
    tval = numerator3 / denominator5

    # compute stats
    p = 2 * (1 - t.cdf(tval, df))

    # should we reject null hypothesis at p < 0.05?
    h = p < 0.05

    return (h, p, ratio)




In [29]:
import random

n_samples=100
y_true = np.arange(n_samples) +1
y_pred = y_true * random.random()
print (y_pred)

print(PitmanMorganTest(y_true, y_pred))

[ 0.95701802  1.91403605  2.87105407  3.82807209  4.78509012  5.74210814
  6.69912617  7.65614419  8.61316221  9.57018024 10.52719826 11.48421628
 12.44123431 13.39825233 14.35527035 15.31228838 16.2693064  17.22632443
 18.18334245 19.14036047 20.0973785  21.05439652 22.01141454 22.96843257
 23.92545059 24.88246862 25.83948664 26.79650466 27.75352269 28.71054071
 29.66755873 30.62457676 31.58159478 32.5386128  33.49563083 34.45264885
 35.40966688 36.3666849  37.32370292 38.28072095 39.23773897 40.19475699
 41.15177502 42.10879304 43.06581106 44.02282909 44.97984711 45.93686514
 46.89388316 47.85090118 48.80791921 49.76493723 50.72195525 51.67897328
 52.6359913  53.59300932 54.55002735 55.50704537 56.4640634  57.42108142
 58.37809944 59.33511747 60.29213549 61.24915351 62.20617154 63.16318956
 64.12020759 65.07722561 66.03424363 66.99126166 67.94827968 68.9052977
 69.86231573 70.81933375 71.77635177 72.7333698  73.69038782 74.64740585
 75.60442387 76.56144189 77.51845992 78.47547794 79.